# KITTIDataset

In [ ]:
from kitti_dataset import *

In [ ]:
dataset = KITTIDataset(data_path="../cached_datasets/KITTI/", split="train")

In [ ]:
dataset[0][0].shape

In [ ]:
dataset[0][1].shape

In [ ]:
4, 376, 1242

In [ ]:
from torchmetrics import JaccardIndex, ConfusionMatrix
import torch
target = torch.randint(0, 18, (4, 376, 1242))
pred = torch.tensor(target)
# print(target)
pred[2:5, 7:13, 9:15] = 19 - pred[2:5, 7:13, 9:15]
jaccard = JaccardIndex(num_classes=19)
jaccard(pred, target)

In [ ]:
pred.shape

In [ ]:
target.shape

In [ ]:
jaccard = JaccardIndex(num_classes=19)
jaccard(pred, target)

In [1]:
import torch

In [2]:
predictions = torch.load("../predictions.pt")

In [3]:
mask = torch.load("../mask.pt")

In [4]:
out = torch.load("../out.pt")

In [ ]:
jaccard = JaccardIndex(num_classes=19)
jaccard(predictions, mask)

In [ ]:
confmat = ConfusionMatrix(num_classes=19)
confmat(predictions, mask)

In [ ]:
predictions.shape, mask.shape

In [ ]:
preds, target, mode = _input_format_classification(predictions, mask, threshold=0.5)

In [ ]:
num_classes = 19
unique_mapping = (mask.view(-1) * num_classes + predictions.view(-1)).to(torch.long)
minlength = num_classes ** 2

In [ ]:
bins = torch.bincount(unique_mapping, minlength=minlength)

In [ ]:
bins.shape

In [ ]:
(mask.view(-1) * num_classes + predictions.view(-1)).shape

In [ ]:
import pandas as pd
x_np = predictions[0].cpu().numpy()
x_df = pd.DataFrame(x_np)
x_df.to_csv('predictions.csv', index=False)

In [ ]:
def iou_score(output, target):
    smooth = 1e-5

    if torch.is_tensor(output):
        output = torch.sigmoid(output).data.cpu().numpy()
    if torch.is_tensor(target):
        target = target.data.cpu().numpy()
    output_ = output > 0.5
    target_ = target > 0.5
    intersection = (output_ & target_).sum()
    union = (output_ | target_).sum()

    return (intersection + smooth) / (union + smooth)

In [ ]:
iou_score(predictions, mask)

In [ ]:
predictions.shape

In [ ]:
mask.shape

In [ ]:
output = torch.sigmoid(predictions).data.cpu().numpy()

In [ ]:
output.shape

In [ ]:
import numpy as np
def metric(probability, truth):
    '''Calculates dice of positive and negative images seperately'''
    '''probability and truth must be torch tensors'''
    base_threshold = 0.5
    batch_size = len(truth)
    with torch.no_grad():
        probability = (probability > base_threshold).float()
        truth = (truth > 0.5).float()

        p = probability.view(batch_size, -1)
        t = truth.view(batch_size, -1)
        assert(p.shape == t.shape)

        intersection = (p*t).sum(-1)
        union = (p+t).sum(-1)

        t_sum = t.sum(-1)
        p_sum = p.sum(-1)
        neg_index = torch.nonzero(t_sum == 0)
        pos_index = torch.nonzero(t_sum >= 1)

        neg = (p_sum == 0).float()
        dice_pos = (2 * intersection) / (union + 1e-7)
        iou_pos = intersection / (union + 1e-7)

        neg = neg[neg_index]
        dice_pos = dice_pos[pos_index]
        iou_pos = iou_pos[pos_index]

        dice = torch.cat([dice_pos, neg])
        iou = torch.cat([iou_pos, neg])

        neg = np.nan_to_num(neg.mean().item(), 0)
        dice_pos = np.nan_to_num(dice_pos.mean().item(), 0)

        dice = dice.mean().item()
        iou = iou.mean().item()

        num_neg = len(neg_index)
        num_pos = len(pos_index)

        dice = {"dice_all": dice}

        # if self.get_class_metric:
        num_classes = probability.shape[1]
        for c in range(num_classes):
            iflat = probability[:, c,...].view(batch_size, -1)
            tflat = truth[:, c,...].view(batch_size, -1)
            intersection = (iflat * tflat).sum()
            dice[str(c)] = ((2. * intersection) / (iflat.sum() + tflat.sum() + 1e-7)).item()

    return iou, dice, neg, dice_pos, num_neg, num_pos

In [ ]:
metric(out, mask)

In [ ]:
out.view(4, -1).shape

In [ ]:
out.view(batch_size, -1)

In [ ]:
import torch
mask = torch.load("../mask.pt")
data = torch.nn.functional.one_hot(mask)

In [ ]:
torch.nn.functional.one_hot((torch.arange(0, 5) % 3))

In [ ]:
# mask.shape

torch.nn.functional.one_hot(mask, num_classes=19)
# categorical[torch.arange(65536), y] = 1
# categorical = torch.reshape(categorical, (256, 256, 10))


In [ ]:
mask

In [44]:
import numpy as np
def compute_ious(pred, label, classes, ignore_index=255, only_present=True):
    pred[label == ignore_index] = 0
    ious = []
    for c in classes:
        label_c = label == c
        if only_present and np.sum(label_c.int().cpu().numpy()) == 0:
            ious.append(nan)
            continue
        pred_c = pred == c
        intersection = np.logical_and(pred_c.cpu(), label_c.cpu()).sum()
        union = np.logical_or(pred_c.cpu(), label_c.cpu()).sum()
        if union != 0:
            ious.append(float(intersection / union))
    # ious = torch.Tensor(ious)
    mean = torch.Tensor(ious).mean()
    return mean if ious else [1]

In [45]:
compute_ious(predictions, mask, classes=[i for i in range(19)], ignore_index=250)

tensor(0.0155)

In [7]:
label = mask
label_c = label == 0

In [12]:
np.sum(label_c.int().cpu().numpy())

496732

In [68]:
def compute_ious(pred, label, classes, ignore_index=255, only_present=True):
    pred[label == ignore_index] = 0
    ious = []
    for c in classes:
        label_c = label == c
        if only_present and np.sum(label_c.int().cpu().numpy()) == 0:
            ious.append(np.nan)
            continue
        pred_c = pred == c
        intersection = np.logical_and(pred_c.cpu(), label_c.cpu()).sum()
        union = np.logical_or(pred_c.cpu(), label_c.cpu()).sum()
        if union != 0:
            ious.append(intersection / union)
    mean = torch.Tensor(ious)
    mean = mean[~mean.isnan()].mean()
    return mean if ious else 0

In [67]:
compute_ious(predictions, mask, classes=[i for i in range(19)], ignore_index=250)

tensor(0.0184)


[tensor(0.0025),
 tensor(0.),
 tensor(0.),
 tensor(0.),
 tensor(0.0001),
 tensor(0.0227),
 tensor(0.0076),
 tensor(0.0100),
 tensor(0.1572),
 tensor(0.0003),
 tensor(0.0845),
 tensor(0.0082),
 tensor(0.0002),
 tensor(0.),
 tensor(0.0005),
 tensor(0.),
 nan,
 nan,
 nan]